In [1]:
import requests
import pandas as pd
import os
from datetime import datetime
import time

In [2]:
def find_class_in_block(block, class_name: str):
    class_title = block.find(class_name)
    return block[class_title : block.find('</a>', class_title)]


def find_obj_by_trigger(cl, trigger, end_trigger):
    len_tr = len(trigger)
    loc_tr = cl.find(trigger)
    return cl[loc_tr+len_tr+1 : cl.find(end_trigger, loc_tr+len_tr+1)]


def string_obj(val):
    return f''' Продавец {val['Продавец']}
    Цена {val['Цена']}
    Бонусы {val['Бонусы']}
    Итого {val['Итог за вычетом бонуса']}
    Ссылка на карточку {val['Ссылка']}
    '''

In [3]:
def get_df_prices_requests(ref_for_category):
    compare_prices = {}
    sep = '%20'
    ref_for_category = sep.join(ref_for_category.split(' '))

    page = 1
    while True:
        
        search_result = requests.get(f'https://megamarket.ru/catalog/page-{page}/?q={sep}{ref_for_category}')#f'{ref_for_category}page-{page}')
        search_result.encoding = 'br'

        ost = search_result.text.split('<div class="sticky-element"><div class="catalog-listing__items catalog-listing__items_divider">')
        if len(ost) < 2:
            break
        ost_ids = ost[1]

        lists_count = ost_ids.split('<nav class="pager catalog-listing__pager">')[1]
        ost_ids = ost_ids.split('<nav class="pager catalog-listing__pager">')[0]
        ids = ost_ids.split('<div id=')[1:]

        for id_pr in ids:
            cl=find_class_in_block(id_pr, '<div class="item-title">')
            product_id = find_obj_by_trigger(cl, 'data-product-id=', '"')
            name = find_obj_by_trigger(cl, 'title=', '"')
            ref = 'https://megamarket.ru'+find_obj_by_trigger(cl, 'href=', '"')

            cl_merch = find_class_in_block(id_pr, '<div class="merchant-info__name"')
            saler = find_obj_by_trigger(cl_merch, '<div class="merchant-info__name"', '<')

            cl_price = find_class_in_block(id_pr, '<div class="item-money"')

            if len(cl_price) <2:
                break
            price = find_obj_by_trigger(cl_price, '<div class="item-price"><span', '<')
            price = int(price[:-2].replace(' ', ''))
            bonus = find_obj_by_trigger(cl_price, '<span class="bonus-amount"', '<')
            try:
                bonus = int(bonus.replace(' ', ''))
            except ValueError:
                bonus = 0

            compare_prices[product_id] = {'id': product_id,
                             'Наименование': name,
                             'Ссылка': ref,
                             'Продавец': saler,
                             'Цена': price,
                             'Бонусы': bonus,
                             'Итог за вычетом бонуса': price-bonus}
        print(page)
        page += 1
    try:
        res = pd.DataFrame().from_dict(compare_prices, orient='index').sort_values(by='Итог за вычетом бонуса')
    except KeyError:
        return 'Попробуйте ввести запрос по другому'
    return f'''Топ 3 варианта:
    1. {string_obj(res.iloc[0])}
    2. {string_obj(res.iloc[1])}
    3. {string_obj(res.iloc[2])}'''

In [230]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time

In [231]:
def get_df_prices_selenium(ref_for_category):
    
    browser = webdriver.Chrome()#ChromeDriverManager().install())
    time.sleep (5) # долго грузится - делаем задержку
    
    sep = '%20'
    ref_for_category = sep.join(ref_for_category.split(' '))
    #for category
    #ref_for_category = 'https://megamarket.ru/catalog/igrovye-pristavki-playstation/set-ps-5/'

    compare_prices = {}
    page = 1
    while True:
        browser.get(f'https://megamarket.ru/catalog/page-{page}/?q={sep}{ref_for_category}')
        elements = browser.find_elements(by=By.CLASS_NAME, value='catalog-item.catalog-item-desktop.ddl_product')
        if len(elements) == 0:
            break

        for element in elements:
            product_id = element.get_attribute('id')
            cl = element.get_attribute('class')
            if 'catalog-item_out-of-stock' in cl:
                break
            title = element.find_element(by=By.CLASS_NAME, value='item-title')
            name = title.text       
            ref = title.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
            saler = element.find_element(by=By.CLASS_NAME, value='merchant-info__name').text
            price = element.find_element(by=By.CLASS_NAME, value='item-price').text
            price = int(price[:-2].replace(' ', ''))
            bonus = 0
            try:
                bonus = element.find_element(by=By.CLASS_NAME, value='bonus-amount').text
                bonus = int(bonus.replace(' ', ''))
            except NoSuchElementException:
                pass

            compare_prices[product_id] = {'id': product_id,
                                          'Наименование': name,
                                          'Ссылка': ref,
                                          'Продавец': saler,
                                          'Цена': price,
                                          'Бонусы': bonus,
                                          'Итог за вычетом бонуса': price-bonus}

        page+=1
        
    res = pd.DataFrame().from_dict(compare_prices, orient='index').sort_values(by='Итог за вычетом бонуса')
    browser.close()

    return f'''Топ 3 варианта:
    1. {string_obj(res.iloc[0])}
    2. {string_obj(res.iloc[1])}
    3. {string_obj(res.iloc[2])}'''

In [10]:
import telebot

API = '#'
bot = telebot.TeleBot(API)

@bot.message_handler(content_types=['text'])
def get_prices(ref):
    bot.reply_to(ref, text='Ожидайте выполнения запроса')
    bot.reply_to(ref, get_df_prices_selenium(ref.text))

In [11]:
bot.infinity_polling()

2023-12-18 13:17:00,505 (__init__.py:960 MainThread) ERROR - TeleBot: "Infinity polling exception: list index out of range"
2023-12-18 13:17:00,507 (__init__.py:962 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/telebot/__init__.py", line 955, in infinity_polling
    self.polling(non_stop=True, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/opt/anaconda3/lib/python3.9/site-packages/telebot/__init__.py", line 1043, in polling
    self.__threaded_polling(non_stop=non_stop, interval=interval, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/opt/anaconda3/lib/python3.9/site-packages/telebot/__init__.py", line 1118, in __threaded_polling
    raise e
  File "/opt/anaconda3/lib/python3.9/site-packages/telebot/__init__.py", line 1074, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/opt/anaconda3/lib/python3.9/site-packages/telebot/util.py", 